# DEF Notebook
This notebook aims to extend the work of the ABC notebook into a Dynamics & Environment Focus (DEF), where the initial ABCs will be turned into movies outlining the evolution of all statistics. The evolutions will both be through time and environment, each showcasing the unique role that phases play in the Large Scale Structure of the Universe.

# Data Import

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as spy

def DataImport(filedir,nx,ny,nz,lx,ly,lz):
    print('Processing amplitudes and phases of file:')
    print(filedir)
    data = np.loadtxt(filedir)
    x_data, y_data, z_data = data[:,0], data[:,1], data[:,2]
    datagrid = np.zeros((nx,ny,nz)) #initialize the data grid
    datagrid, edges = np.histogramdd(np.vstack([x_data,y_data,z_data]).transpose(),bins=(nx,ny,nz),range=((0.,lx),(0.,ly),(0.,lz)))
    mean_density = np.float(len(x_data))/np.float(nx*ny*nz)
    overdensities = datagrid/mean_density-1 +0j
    FFTfield = np.fft.fftn(overdensities) #Perform a Fourier Transform on the density field
    amplitudes = np.sqrt(FFTfield.real**2 + FFTfield.imag**2)
    phases = np.arctan2(FFTfield.imag, FFTfield.real)
    print('Done!')
    return overdensities, amplitudes, phases

#GRF Generation
#L_array = np.linspace(0,field_length,resolution) #Array for values of length
#GRFgrid=np.zeros((resolution, resolution, resolution),dtype=complex) #Initial state of 3D grid (zeros)
#for i in range(0,resolution): #For loop to create a 3D grid
#    for j in range(0,resolution):
#        for k in range(0,resolution):
#            GRFgrid[i,j,k] = 0.03*np.random.normal(loc = 0.000301, scale=2668.74773828) #Assigns each pixel a GRF value (standardised to GiggleZ statistics)
#FFTgrid = np.fft.fftn(GRFgrid) #Perform a Fourier Transform on the density field
#GRFamplitudes = np.sqrt(FFTgrid.real**2 + FFTgrid.imag**2)
#GRFphases = np.arctan2(FFTgrid.imag, FFTgrid.real)

In [9]:
def PowerSpectra(overdensities, resolution, amps, phases, k_tot, redshift):
    print('Plotting Power Spectra... ({0})'.format(redshift))
    binnum = 30 #Number of total bins
    kmin = np.min(k_tot)
    kmax = np.max(k_tot)
    dk = (kmax-kmin)/binnum #Width of a bin
    ikbin = np.digitize(k_tot,np.linspace(kmin,kmax,binnum+1)) #Bin the values of k total into binnum
    nmodes,pk,pk_err = np.zeros(binnum,dtype=int),np.full(binnum,-1.),np.full(binnum,-1.) #initialize nmodes and power spectrum
    amp_pwr = amps**2
    phase_pwr = phases**2
    binmids=np.zeros(binnum)
    binmids = np.linspace(kmin+0.5*dk, kmax-0.5*dk, binnum)

    sig = np.std(overdensities)
    shotnoise = resolution**3*sig**2
    sigma_phase = 0#np.std(phases)
    phasenoise = sigma_phase**2

    #Initialize other power spectra arrays
    phk, phk_err = np.full(binnum,-1.),np.full(binnum,-1.)
    for ik in range(binnum):
        nmodes[ik] = int(np.sum(np.array([ikbin == ik+1])))
        if (nmodes[ik] > 0):
            #GiggleZ Amp power spectrum
            pk[ik] = np.mean(amp_pwr[ikbin == ik+1])
            pk_err[ik] = np.std(amp_pwr[ikbin == ik+1])/np.sqrt(nmodes[ik]/2)
            #GiggleZ Phase power spectrum
            phk[ik] = np.mean(phase_pwr[ikbin == ik+1])
            phk_err[ik] = np.std(phase_pwr[ikbin == ik+1])/np.sqrt(nmodes[ik]/2)

    plt.close("all")

    plt.figure('AmpPwr')
    plt.title('Amplitude Power Spectra ({0}) (Noise Reduced)'.format(redshift))
    plt.xlabel('$k_{total}$')
    plt.ylabel('Amplitude Power')
    plt.scatter(binmids, pk-shotnoise, s=8, color='blue')
    plt.errorbar(binmids,pk-shotnoise,pk_err,color='blue')
    plt.savefig('C:/Users/Simon/Documents/Honours/Phases of the Universe/AmpPwr/f_{0}.png'.format(redshift), bbox_inches='tight')

    plt.figure('PhasePwr')
    plt.title('Phase Power Spectra ({0}) (Noise Reduced)'.format(redshift))
    plt.xlabel('$k_{total}$')
    plt.ylabel('Phase Power')
    plt.ylims([0, 2*np.pi])
    plt.scatter(binmids, phk-phasenoise, s=8, color='blue')
    plt.errorbar(binmids,phk-phasenoise,phk_err,color='blue')
    plt.savefig('C:/Users/Simon/Documents/Honours/Phases of the Universe/PhasePwr/f_{0}.png'.format(redshift), bbox_inches='tight')

    print('Done!')
    
def Main():
    resolution = 256 #Dictates the number of bins in x,y,z (Use powers of 2)
    field_length = 1000 #The length of x,y,z in Mpc/h
    nx,ny,nz = resolution,resolution,resolution #number of bins in x, y and z. Stick to powers of 2!
    lx,ly,lz = field_length,field_length,field_length #needs units (Mpc/h)! #length of each dimension
    
    dx,dy,dz = lx/nx,ly/ny,lz/nz #length of each bin
    kx = 2.*np.pi*np.fft.fftfreq(nx,d=lx/nx) 
    ky = 2.*np.pi*np.fft.fftfreq(ny,d=ly/ny)
    kz = 2.*np.pi*np.fft.fftfreq(nz,d=lz/nz)
    k_tot = np.sqrt(kx[:,np.newaxis,np.newaxis]**2 + ky[np.newaxis,:,np.newaxis]**2 + kz[np.newaxis,np.newaxis,:]**2)
    
    datasets = ['z0pt000.dat', 'z0pt208.dat', 'z0pt299.dat', 'z0pt408.dat',
           'z0pt509.dat', 'z0pt593.dat', 'z0pt687.dat', 'z0pt721.dat',
           'z0pt791.dat', 'z0pt989.dat', 'z1pt224.dat', 'z1pt386.dat',
           'z1pt504.dat', 'z2pt070.dat']
    
    redshifts = ['z=0.000', 'z=0.208', 'z=0.299', 'z=0.408', 
                 'z=0.509', 'z=0.593', 'z=0.687', 'z=0.721', 
                 'z=0.791', 'z=0.989', 'z=1.224', 'z=1.386', 'z=1.504', 'z=2.070']
    
    for dataset in range(len(datasets)):
        ODs, amps, phases = 0, 0, 0
        ODs, amps, phases = DataImport('C:/Users/Simon/Documents/Honours/Phases of the Universe/GiggleZ redshift data/{0}'.format(datasets[dataset]),nx,ny,nz,lx,ly,lz) #Returns amplitudes and phases of dataset. Also saves overdensity field image.
        PowerSpectra(ODs, resolution, amps, phases, k_tot, redshifts[dataset]) #Creates Power spectra of current dataset and saves comparative plots
    #[for loop ends]
    #MoVis()

In [7]:
Main()

Processing amplitudes and phases of file:
C:/Users/Simon/Documents/Honours/Phases of the Universe/GiggleZ redshift data/z0pt000.dat
Done!
Plotting Power Spectra... (z=0.000)
Done!
Processing amplitudes and phases of file:
C:/Users/Simon/Documents/Honours/Phases of the Universe/GiggleZ redshift data/z0pt208.dat
Done!
Plotting Power Spectra... (z=0.208)
Done!
Processing amplitudes and phases of file:
C:/Users/Simon/Documents/Honours/Phases of the Universe/GiggleZ redshift data/z0pt299.dat
Done!
Plotting Power Spectra... (z=0.299)
Done!
Processing amplitudes and phases of file:
C:/Users/Simon/Documents/Honours/Phases of the Universe/GiggleZ redshift data/z0pt408.dat
Done!
Plotting Power Spectra... (z=0.408)
Done!
Processing amplitudes and phases of file:
C:/Users/Simon/Documents/Honours/Phases of the Universe/GiggleZ redshift data/z0pt509.dat
Done!
Plotting Power Spectra... (z=0.509)
Done!
Processing amplitudes and phases of file:
C:/Users/Simon/Documents/Honours/Phases of the Universe/Gi